In [18]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [19]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=abm_14_2_0_reporting;'
                    'Trusted_Connection=yes;')

query ='''-- connect to ddamwsql2016
SELECT [scenario_id]
      ,[measure]
      ,[metric]
      ,ROUND([value]*100,1) as [rounded_val] -- change this depending on value, i.e. if percent or other value
      ,[value] as [og_val]
      ,[updated_by]
      ,[updated_date]
  FROM [abm_14_2_0_reporting].[rp_2021].[results]
  WHERE scenario_id = 759
  AND measure like 'M-1%'
  --AND metric like 'Population Access%' -- change this depending on the measure'''

sql_data =  pd.read_sql_query(query, conn)

In [20]:
lst = list(sql_data['metric'])

lst = [s.split('-') for s in lst]

#print(lst)

In [21]:
first_values = [inner_lst[0] if inner_lst[0] != 'Non' else inner_lst[0]+'-'+inner_lst[1] for inner_lst in lst]
second_values = [inner_lst[1] if inner_lst[0] != 'Non' else inner_lst[2] for inner_lst in lst]

In [22]:
sql_data['population'] = first_values
sql_data['mode'] = second_values

In [23]:
sql_data

,scenario_id,measure,metric,rounded_val,og_val,updated_by,updated_date,population,mode
0,759,M-1-a,Low Income Access Pct - Bike,96.2,0.962315,SANDAGNET\ndh,2022-12-14 13:09:00,Low Income Access Pct,Bike
1,759,M-1-a,Low Income Access Pct - Bike + Walk + MM + MT,96.2,0.962315,SANDAGNET\ndh,2022-12-14 13:10:00,Low Income Access Pct,Bike + Walk + MM + MT
2,759,M-1-a,Low Income Access Pct - Drive Alone,98.5,0.984988,SANDAGNET\ndh,2022-12-14 13:11:00,Low Income Access Pct,Drive Alone
3,759,M-1-a,Low Income Access Pct - Transit - Speed One,72.8,0.727992,SANDAGNET\ndh,2022-12-14 13:11:00,Low Income Access Pct,Transit
4,759,M-1-a,Low Income Access Pct - Walk,78.8,0.788350,SANDAGNET\ndh,2022-12-14 13:11:00,Low Income Access Pct,Walk
...,...,...,...,...,...,...,...,...,...
191,759,M-1-c - Mobility Hubs,Non-Senior Access Pct - Transit - Speed One,96.1,0.961062,SANDAGNET\ndh,2022-12-14 13:23:00,Non-Senior Access Pct,Transit
192,759,M-1-c - Mobility Hubs,Population Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:23:00,Population Access Pct,Drive Alone
193,759,M-1-c - Mobility Hubs,Population Access Pct - Transit - Speed One,96.1,0.961038,SANDAGNET\ndh,2022-12-14 13:23:00,Population Access Pct,Transit
194,759,M-1-c - Mobility Hubs,Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:23:00,Senior Access Pct,Drive Alone


In [24]:
sql_data['mode'] = sql_data['mode'].replace({' Bike':'Bike', 
                                            ' Bike + Walk + MM + MT':'Walk, Bike, Micromobility, Microtransit',
                                            ' Drive Alone':'Driving (drive alone)',
                                            ' Transit ': 'Transit - Accessed by Walk and or Flexible Fleet - Speed One',
                                            ' Walk':'Walk',
                                            ' Walk + MM + MT':'Walk, Micromobility, Microtransit'
                                            })

In [25]:
output_rank= {'Bike':2, 
                                            'Walk, Bike, Micromobility, Microtransit':4,
                                            'Driving (drive alone)':6,
                                            'Transit - Accessed by Walk and or Flexible Fleet - Speed One':5,
                                            'Walk':1,
                                            'Walk, Micromobility, Microtransit':3
                                            }

In [26]:
sql_data['output_ranking'] = [output_rank[mode] for mode in sql_data['mode']]

In [27]:
final_output = sql_data.sort_values('output_ranking')

In [28]:
set(final_output['measure'])

{'M-1-a',
 'M-1-a - Mobility Hubs',
 'M-1-b',
 'M-1-b - Mobility Hubs',
 'M-1-c',
 'M-1-c - Mobility Hubs'}

In [29]:
set(final_output['population'])

{'Low Income Access Pct ',
 'Minority Access Pct ',
 'Non-Low Income Access Pct ',
 'Non-Minority Access Pct ',
 'Non-Senior Access Pct ',
 'Population Access Pct ',
 'Senior Access Pct '}

In [30]:
# measures = ['M-1-a',
#  'M-1-a - Mobility Hubs',
#  'M-1-b',
#  'M-1-b - Mobility Hubs',
#  'M-1-c',
#  'M-1-c - Mobility Hubs']

measures = ['M-1-a','M-1-b','M-1-c']



# pops = ['Low Income Access Pct ',
#  'Minority Access Pct ',
#  'Non-Low Income Access Pct ',
#  'Non-Minority Access Pct ',
#  'Non-Senior Access Pct ',
#  'Population Access Pct ',
#  'Senior Access Pct ']

pops = ['Low Income Access Pct ','Non-Low Income Access Pct ','Minority Access Pct ','Non-Minority Access Pct ','Senior Access Pct ','Non-Senior Access Pct ']

In [31]:
for measure in measures:
    for ind_pop in pops:
        print(measure + ' - ' + ind_pop + ' -  Regional')
        df = final_output[final_output['measure'] == measure]
        df = df[df['population'] == ind_pop]
        print(df[['mode', 'rounded_val']])

        print(measure + ' - ' + ind_pop + ' -  Mohub')
        df = final_output[final_output['measure'] == measure+' - Mobility Hubs']
        df = df[df['population'] == ind_pop]
        print(df[['mode', 'rounded_val']])

        # print(measure)
        # print(measure+' - Mobility Hubs')
        # print(ind_pop)



        # print(measure + ' - ' + ind_pop)
        # df = final_output[final_output['measure'] == measure]
        # df = df[df['population'] == ind_pop]
        # print(df[['mode', 'rounded_val']])

M-1-a - Low Income Access Pct  -  Regional
                                                mode  rounded_val
4                                               Walk         78.8
0                                               Bike         96.2
5                  Walk, Micromobility, Microtransit         81.1
1            Walk, Bike, Micromobility, Microtransit         96.2
3  Transit - Accessed by Walk and or Flexible Fle...         72.8
2                              Driving (drive alone)         98.5
M-1-a - Low Income Access Pct  -  Mohub
                                                 mode  rounded_val
46                                               Walk         95.1
42                                               Bike        100.0
47                  Walk, Micromobility, Microtransit         98.5
43            Walk, Bike, Micromobility, Microtransit        100.0
45  Transit - Accessed by Walk and or Flexible Fle...         91.2
44                              Driving (drive alone)

In [32]:
df = final_output[final_output['measure'] == 'M-1-c - Mobility Hubs']
df = df[df['population'] == 'Senior Access Pct ']

df[['mode', 'rounded_val']]

,mode,rounded_val
195,Transit - Accessed by Walk and or Flexible Fle...,96.1
194,Driving (drive alone),100.0


In [33]:
final_output

,scenario_id,measure,metric,rounded_val,og_val,updated_by,updated_date,population,mode,output_ranking
94,759,M-1-b,Minority Access Pct - Walk,56.2,0.562211,SANDAGNET\ndh,2022-12-14 13:17:00,Minority Access Pct,Walk,1
70,759,M-1-a - Mobility Hubs,Non-Senior Access Pct - Walk,93.1,0.930561,SANDAGNET\ndh,2022-12-14 13:15:00,Non-Senior Access Pct,Walk,1
22,759,M-1-a,Non-Minority Access Pct - Walk,67.0,0.669510,SANDAGNET\ndh,2022-12-14 13:11:00,Non-Minority Access Pct,Walk,1
118,759,M-1-b,Population Access Pct - Walk,52.7,0.527154,SANDAGNET\ndh,2022-12-14 13:17:00,Population Access Pct,Walk,1
64,759,M-1-a - Mobility Hubs,Non-Minority Access Pct - Walk,93.1,0.931041,SANDAGNET\ndh,2022-12-14 13:15:00,Non-Minority Access Pct,Walk,1
...,...,...,...,...,...,...,...,...,...,...
164,759,M-1-b - Mobility Hubs,Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:18:00,Senior Access Pct,Driving (drive alone),6
68,759,M-1-a - Mobility Hubs,Non-Senior Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:14:00,Non-Senior Access Pct,Driving (drive alone),6
168,759,M-1-c,Low Income Access Pct - Drive Alone,100.0,0.999956,SANDAGNET\ndh,2022-12-14 13:21:00,Low Income Access Pct,Driving (drive alone),6
56,759,M-1-a - Mobility Hubs,Non-Low Income Access Pct - Drive Alone,100.0,1.000000,SANDAGNET\ndh,2022-12-14 13:14:00,Non-Low Income Access Pct,Driving (drive alone),6
